In [1]:
import datetime
import numpy as np
import pandas as pd
import json
import requests
from datetime import date

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://en.shmet.com/api/rest/enweb/spot/getChartPrices?scId=811&startDate=2020-01-01&endDate=2023-07-28&_=1690186841910'

response = requests.get(url)


In [3]:
if response.status_code == 200:  # Проверка успешности запроса
    data = response.json()  # Получение JSON-данных из ответа

    with open("./data/shmet.json", "w") as file:  # Открытие файла для записи
        json.dump(data, file)  # Запись JSON-данных в файл
        print("JSON-файл успешно сохранен.")
else:
    print("Не удалось получить JSON-данные.")


JSON-файл успешно сохранен.


In [4]:
temp = pd.read_json('./data/shmet.json')['data']['spotPrices']
data = pd.DataFrame(temp)
data.head()


,dateTime,price,unit
0,1690515000000,68950,Yuan/MT
1,1690428600000,69460,Yuan/MT
2,1690342200000,69200,Yuan/MT
3,1690255800000,68980,Yuan/MT
4,1690169400000,68520,Yuan/MT


In [5]:
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


In [6]:
data['dateTime'] = data['dateTime'].apply(date_format)
data.rename({'dateTime': 'date'}, axis=1, inplace=True)
data['date'] = pd.to_datetime(data['date']).dt.date
data.head()

,date,price,unit
0,2023-07-28,68950,Yuan/MT
1,2023-07-27,69460,Yuan/MT
2,2023-07-26,69200,Yuan/MT
3,2023-07-25,68980,Yuan/MT
4,2023-07-24,68520,Yuan/MT


In [7]:
data.to_excel('./data/shmet/shmet_historical_catode_raw.xlsx', index=False)

In [8]:
usd_raw = pd.read_excel('./data/shmet/USD_full.xlsx')
usd_raw.rename({'curs':'USD', 'data':'date'}, axis=1, inplace=True)

yn_raw = pd.read_excel('./data/shmet/YN_full.xlsx')
yn_raw.rename({'curs':'Yuan', 'data':'date'}, axis=1, inplace=True)

yn_raw['Yuan'] = yn_raw['Yuan'] / yn_raw['nominal']

usd = usd_raw[['date', 'USD']]
yn = yn_raw[['date', 'Yuan']]

currency = usd.merge(yn)
currency['date'] = pd.to_datetime(currency['date']).dt.date
currency.head()



,date,USD,Yuan
0,2023-07-28,90.0225,12.5897
1,2023-07-27,90.0468,12.5766
2,2023-07-26,90.0945,12.5988
3,2023-07-25,90.4890,12.5536
4,2023-07-22,90.3846,12.5784


In [9]:
data_currency = data.merge(currency)
data_currency['price'] = data_currency['price'].apply(pd.to_numeric)
display(data_currency.head())
data_currency.info()

,date,price,unit,USD,Yuan
0,2023-07-28,68950,Yuan/MT,90.0225,12.5897
1,2023-07-27,69460,Yuan/MT,90.0468,12.5766
2,2023-07-26,69200,Yuan/MT,90.0945,12.5988
3,2023-07-25,68980,Yuan/MT,90.4890,12.5536
4,2023-07-21,68670,Yuan/MT,90.8545,12.6455


<class 'pandas.core.frame.DataFrame'>
Int64Index: 656 entries, 0 to 655
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    656 non-null    object 
 1   price   656 non-null    int64  
 2   unit    656 non-null    object 
 3   USD     656 non-null    float64
 4   Yuan    656 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 30.8+ KB


In [10]:
data_currency['price_usd'] = round(
    (data_currency['Yuan'] / data_currency['USD']) * data_currency['price'], 3)

data_currency['rate'] = round(data_currency['Yuan'] / data_currency['USD'], 4)
data_currency.head()


,date,price,unit,USD,Yuan,price_usd,rate
0,2023-07-28,68950,Yuan/MT,90.0225,12.5897,9642.698,0.1399
1,2023-07-27,69460,Yuan/MT,90.0468,12.5766,9701.296,0.1397
2,2023-07-26,69200,Yuan/MT,90.0945,12.5988,9676.917,0.1398
3,2023-07-25,68980,Yuan/MT,90.4890,12.5536,9569.642,0.1387
4,2023-07-21,68670,Yuan/MT,90.8545,12.6455,9557.771,0.1392


In [11]:
data_currency.to_excel('./data/shmet/shmet_historical_catode.xlsx', index=False)

---

In [ ]:
data_currency['date'] = pd.to_datetime(data_currency['date'])
data_currency['month'] = data_currency['date'].dt.month
data_currency['year'] = data_currency['date'].dt.year

mean_price = data_currency.groupby(by=['year', 'month'], as_index=False)['price'].mean()
mean_price.query("year == 2023")

In [ ]:
df = pd.read_excel('./data/LME_db.xlsx', index_col=0)
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
lme_mean = df.groupby(by=['year', 'month'], as_index=False)['copper'].mean()
lme_mean.query("year == 2023")

In [14]:
url_2 = 'https://www.shmet.com/api/rest/datacenter/getHistoryPrices?startDate=2020-01-01&endDate=2023-07-25&lcfgId=4065&_=1690275161987'
response = requests.get(url_2)

if response.status_code == 200:  # Проверка успешности запроса
    data = response.json()  # Получение JSON-данных из ответа

    with open("./data/shmet_2.json", "w") as file:  # Открытие файла для записи
        json.dump(data, file)  # Запись JSON-данных в файл
        print("JSON-файл успешно сохранен.")
else:
    print("Не удалось получить JSON-данные.")
    



JSON-файл успешно сохранен.


In [15]:
temp = pd.read_json('./data/shmet_2.json')
data = pd.DataFrame(pd.DataFrame(temp)['data']['prices'])
data


,dateTime,publishDateTime,high,low,middle,change,rate,unit
0,1690255800000,2023-07-25 11:30:00,69130,68830,68980,+460,+0.67%,元/吨


In [130]:
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def shmet_parser():
    day = date.today()
    today = day.strftime('%Y-%m-%d')
    url = f'https://en.shmet.com/api/rest/enweb/spot/getChartPrices?scId=811&startDate={day}&endDate={day}&_=1690186841910'

    response = requests.get(url)
    if response.status_code == 200:  # Проверка успешности запроса
        data = response.json()  # Получение JSON-данных из ответа

    with open("./data/shmet.json", "w") as file:  # Открытие файла для записи
        json.dump(data, file)  # Запись JSON-данных в файл

    temp = pd.read_json('./data/shmet.json')['data']['spotPrices']
    data = pd.DataFrame(temp)

    data['dateTime'] = data['dateTime'].apply(date_format)
    data.rename({'dateTime': 'date'}, axis=1, inplace=True)
    data['date'] = pd.to_datetime(data['date']).dt.date
    
    

    return data

test_raw = shmet_parser()

In [131]:
day = date.today()
today = day.strftime('%Y-%m-%d')
today

'2023-07-28'

In [132]:
test_raw['date'] = pd.to_datetime(test_raw['date'])
test_raw

,date,price,unit
0,2023-07-28,68950,Yuan/MT


In [133]:
currency_df = pd.read_excel(
        '../parser_beta/data/cb_curr.xlsx', index_col=0)

temp_cur = currency_df.query("Id == 'USD' or Id == 'China_Yuan'")

temp_cur['Value'] = temp_cur['Value']/temp_cur['Nominal']

temp_cur = temp_cur[temp_cur['Date'] ==
                today].reset_index().drop(['index', 'Nominal'], axis=1)

pivot_table = temp_cur.pivot(index='Date', columns='Id', values='Value')

pivot_table = pivot_table.reset_index().rename(columns={'Date':'date', 'China_Yuan':'Yuan'})
pivot_table

Id,date,Yuan,USD
0,2023-07-28,12.5897,90.0225


In [134]:
shmet_raw = test_raw.merge(right=pivot_table, right_on='date', left_on='date')
shmet_raw['price'] = pd.to_numeric(shmet_raw['price'])
shmet_raw['rate'] = shmet_raw['Yuan'] / shmet_raw['USD']
shmet_raw['price_usd'] = shmet_raw['price'] * shmet_raw['rate']
shmet_raw = shmet_raw.reindex(columns=['date', 'price', 'unit', 'USD', 'Yuan', 'price_usd', 'rate'])
shmet_raw


,date,price,unit,USD,Yuan,price_usd,rate
0,2023-07-28,68950,Yuan/MT,90.0225,12.5897,9642.698381,0.139851


In [135]:
df = pd.read_excel('./data/shmet/shmet_test.xlsx')
df

,date,price,unit,USD,Yuan,price_usd,rate
0,2023-07-27,69460,Yuan/MT,90.0468,12.57660,9701.296,0.1397
1,2023-07-26,69200,Yuan/MT,90.0945,12.59880,9676.917,0.1398
2,2023-07-25,68980,Yuan/MT,90.4890,12.55360,9569.642,0.1387
3,2023-07-21,68670,Yuan/MT,90.8545,12.64550,9557.771,0.1392
4,2023-07-20,68650,Yuan/MT,91.2046,12.63030,9506.868,0.1385
...,...,...,...,...,...,...,...
650,2020-01-17,48930,Yuan/MT,61.5694,8.94215,7106.442,0.1452
651,2020-01-16,48950,Yuan/MT,61.4328,8.92024,7107.697,0.1452
652,2020-01-15,49060,Yuan/MT,61.4140,8.91971,7125.427,0.1452
653,2020-01-14,48990,Yuan/MT,60.9474,8.84000,7105.662,0.1450


In [136]:
shmet_data = pd.concat([shmet_raw, df])
shmet_data

,date,price,unit,USD,Yuan,price_usd,rate
0,2023-07-28,68950,Yuan/MT,90.0225,12.58970,9642.698381,0.139851
0,2023-07-27,69460,Yuan/MT,90.0468,12.57660,9701.296000,0.139700
1,2023-07-26,69200,Yuan/MT,90.0945,12.59880,9676.917000,0.139800
2,2023-07-25,68980,Yuan/MT,90.4890,12.55360,9569.642000,0.138700
3,2023-07-21,68670,Yuan/MT,90.8545,12.64550,9557.771000,0.139200
...,...,...,...,...,...,...,...
650,2020-01-17,48930,Yuan/MT,61.5694,8.94215,7106.442000,0.145200
651,2020-01-16,48950,Yuan/MT,61.4328,8.92024,7107.697000,0.145200
652,2020-01-15,49060,Yuan/MT,61.4140,8.91971,7125.427000,0.145200
653,2020-01-14,48990,Yuan/MT,60.9474,8.84000,7105.662000,0.145000


In [15]:
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def shmet_parser():
    day = date.today()
    today = day.strftime('%Y-%m-%d')
    url = f'https://en.shmet.com/api/rest/enweb/spot/getChartPrices?scId=811&startDate={day}&endDate={day}&_=1690186841910'

    response = requests.get(url)
    if response.status_code == 200:  # Проверка успешности запроса
        data = response.json()  # Получение JSON-данных из ответа

    with open("./data/shmet.json", "w") as file:  # Открытие файла для записи
        json.dump(data, file)  # Запись JSON-данных в файл

    temp = pd.read_json('./data/shmet.json')['data']['spotPrices']
    data = pd.DataFrame(temp)

    data['dateTime'] = data['dateTime'].apply(date_format)
    data.rename({'dateTime': 'date'}, axis=1, inplace=True)
    data['date'] = pd.to_datetime(data['date']).dt.date
    data['date'] = pd.to_datetime(data['date'])

    currency_df = pd.read_excel(
        '../parser_beta/data/cb_curr.xlsx', index_col=0)

    temp_cur = currency_df.query("Id == 'USD' or Id == 'China_Yuan'")

    temp_cur['Value'] = temp_cur['Value']/temp_cur['Nominal']

    temp_cur = temp_cur[temp_cur['Date'] ==
                        today].reset_index().drop(['index', 'Nominal'], axis=1)

    pivot_table = temp_cur.pivot(index='Date', columns='Id', values='Value')

    pivot_table = pivot_table.reset_index().rename(
        columns={'Date': 'date', 'China_Yuan': 'Yuan'})

    shmet_raw = data.merge(right=pivot_table, right_on='date', left_on='date')
    shmet_raw['price'] = pd.to_numeric(shmet_raw['price'])
    shmet_raw['rate'] = shmet_raw['Yuan'] / shmet_raw['USD']
    shmet_raw['price_usd'] = shmet_raw['price'] * shmet_raw['rate']
    shmet_raw = shmet_raw.reindex(
        columns=['date', 'price', 'unit', 'USD', 'Yuan', 'price_usd', 'rate'])

    df = pd.read_excel('./data/shmet_historical.xlsx')
    df.drop_duplicates(inplace=True)

    shmet_data = pd.concat([shmet_raw, df])
    shmet_data.drop_duplicates(inplace=True)
    
    shmet_data.to_excel('./data/shmet_historical.xlsx', index=False)
    

    return shmet_data


In [26]:
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def shmet_parser():
    def shmet_day():
        day = date.today()
        today = day.strftime('%Y-%m-%d')
        url = f'https://en.shmet.com/api/rest/enweb/spot/getChartPrices?scId=811&startDate={day}&endDate={day}&_=1690186841910'

        response = requests.get(url)
        if response.status_code == 200:  # Проверка успешности запроса
            data = response.json()  # Получение JSON-данных из ответа

        with open("./data/shmet.json", "w") as file:  # Открытие файла для записи
            json.dump(data, file)  # Запись JSON-данных в файл

        temp = pd.read_json('./data/shmet.json')['data']['spotPrices']
        data = pd.DataFrame(temp)

        data['dateTime'] = data['dateTime'].apply(date_format)
        data.rename({'dateTime': 'date'}, axis=1, inplace=True)
        data['date'] = pd.to_datetime(data['date']).dt.date
        data['date'] = pd.to_datetime(data['date'])

        currency_df = pd.read_excel(
            '../parser_beta/data/cb_curr.xlsx', index_col=0)

        temp_cur = currency_df.query("Id == 'USD' or Id == 'China_Yuan'")

        temp_cur['Value'] = temp_cur['Value']/temp_cur['Nominal']

        temp_cur = temp_cur[temp_cur['Date'] ==
                            today].reset_index().drop(['index', 'Nominal'], axis=1)

        pivot_table = temp_cur.pivot(
            index='Date', columns='Id', values='Value')

        pivot_table = pivot_table.reset_index().rename(
            columns={'Date': 'date', 'China_Yuan': 'Yuan'})

        shmet_raw = data.merge(
            right=pivot_table, right_on='date', left_on='date')
        shmet_raw['price'] = pd.to_numeric(shmet_raw['price'])
        shmet_raw['rate'] = shmet_raw['Yuan'] / shmet_raw['USD']
        shmet_raw['price_usd'] = shmet_raw['price'] * shmet_raw['rate']
        shmet_raw = shmet_raw.reindex(
            columns=['date', 'price', 'unit', 'USD', 'Yuan', 'price_usd', 'rate'])

        return shmet_raw

    df = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
    df.drop_duplicates(inplace=True)
    
    try:
        day_row = shmet_day()
        shmet_db = pd.concat([day_row, df], axis=0)#, ignore_index=True)

    except IndexError:
        print('Shmet response is empty, please check the source.')
    
    shmet_db = shmet_db.reset_index()
    shmet_db = shmet_db.drop('index', axis=1)
    
    shmet_db.drop_duplicates(inplace=True)
    
    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
        date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        shmet_db.to_excel(writer, sheet_name='SHMET')
        
    final = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
    final.drop_duplicates(inplace=True)
    
    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
        date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        final.to_excel(writer, sheet_name='SHMET')

    return final




In [29]:
shmet_parser()

,date,price,unit,USD,Yuan,price_usd,rate
0,2023-07-28,68950,Yuan/MT,90.0225,12.58970,9642.698381,0.139851
2,2023-07-27,69460,Yuan/MT,90.0468,12.57660,9701.296000,0.139700
3,2023-07-26,69200,Yuan/MT,90.0945,12.59880,9676.917000,0.139800
4,2023-07-25,68980,Yuan/MT,90.4890,12.55360,9569.642000,0.138700
5,2023-07-21,68670,Yuan/MT,90.8545,12.64550,9557.771000,0.139200
...,...,...,...,...,...,...,...
652,2020-01-17,48930,Yuan/MT,61.5694,8.94215,7106.442000,0.145200
653,2020-01-16,48950,Yuan/MT,61.4328,8.92024,7107.697000,0.145200
654,2020-01-15,49060,Yuan/MT,61.4140,8.91971,7125.427000,0.145200
655,2020-01-14,48990,Yuan/MT,60.9474,8.84000,7105.662000,0.145000


In [25]:
df = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
df.drop_duplicates()

,date,price,unit,USD,Yuan,price_usd,rate
0,2023-07-28,68950,Yuan/MT,90.0225,12.58970,9642.698381,0.139851
2,2023-07-27,69460,Yuan/MT,90.0468,12.57660,9701.296000,0.139700
3,2023-07-26,69200,Yuan/MT,90.0945,12.59880,9676.917000,0.139800
4,2023-07-25,68980,Yuan/MT,90.4890,12.55360,9569.642000,0.138700
5,2023-07-21,68670,Yuan/MT,90.8545,12.64550,9557.771000,0.139200
...,...,...,...,...,...,...,...
652,2020-01-17,48930,Yuan/MT,61.5694,8.94215,7106.442000,0.145200
653,2020-01-16,48950,Yuan/MT,61.4328,8.92024,7107.697000,0.145200
654,2020-01-15,49060,Yuan/MT,61.4140,8.91971,7125.427000,0.145200
655,2020-01-14,48990,Yuan/MT,60.9474,8.84000,7105.662000,0.145000
